In [2]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

In [3]:
import sys
import gzip
import numpy as np
import pandas as pd
import dill
from sklearn import neighbors
from matplotlib import pylab as plt
from sklearn import pipeline  
from sklearn import linear_model, utils, preprocessing

In [18]:
import gzip
# read the entire file into a python array
with gzip.open('yelp_train_academic_dataset_business.json.gz', 'rb') as f:
    data = f.readlines()

data[:2]

['{"business_id": "vcNAWiLM4dR7D2nwwJ7nCA", "full_address": "4840 E Indian School Rd\\nSte 101\\nPhoenix, AZ 85018", "hours": {"Tuesday": {"close": "17:00", "open": "08:00"}, "Friday": {"close": "17:00", "open": "08:00"}, "Monday": {"close": "17:00", "open": "08:00"}, "Wednesday": {"close": "17:00", "open": "08:00"}, "Thursday": {"close": "17:00", "open": "08:00"}}, "open": true, "categories": ["Doctors", "Health & Medical"], "city": "Phoenix", "review_count": 7, "name": "Eric Goldberg, MD", "neighborhoods": [], "longitude": -111.98375799999999, "state": "AZ", "stars": 3.5, "latitude": 33.499313000000001, "attributes": {"By Appointment Only": true}, "type": "business"}\n',
 '{"business_id": "JwUE5GmEO-sH1FuwJgKBlQ", "full_address": "6162 US Highway 51\\nDe Forest, WI 53532", "hours": {}, "open": true, "categories": ["Restaurants"], "city": "De Forest", "review_count": 26, "name": "Pine Cone Restaurant", "neighborhoods": [], "longitude": -89.335843999999994, "state": "WI", "stars": 4.0,

In [19]:
# remove the trailing "\n" from each line
data = map(lambda x: x.rstrip(), data)

In [20]:
import pandas as pd
# each element of 'data' is an individual JSON object.
# i want to convert it into an *array* of JSON objects
# which, in and of itself, is one large JSON object
# basically... add square brackets to the beginning
# and end, and have all the individual business JSON objects
# separated by a comma
data_json_str = "[" + ','.join(data) + "]"

# now, load it into pandas
df = pd.read_json(data_json_str)


In [21]:
df.head()

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
0,{u'By Appointment Only': True},vcNAWiLM4dR7D2nwwJ7nCA,"[Doctors, Health & Medical]",Phoenix,"4840 E Indian School Rd\nSte 101\nPhoenix, AZ ...","{u'Thursday': {u'close': u'17:00', u'open': u'...",33.499313,-111.983758,"Eric Goldberg, MD",[],True,7,3.5,AZ,business
1,"{u'Take-out': True, u'Price Range': 1, u'Outdo...",JwUE5GmEO-sH1FuwJgKBlQ,[Restaurants],De Forest,"6162 US Highway 51\nDe Forest, WI 53532",{},43.238893,-89.335844,Pine Cone Restaurant,[],True,26,4.0,WI,business
2,"{u'Take-out': True, u'Outdoor Seating': False,...",uGykseHzyS5xAMWoN6YUqA,"[American (Traditional), Restaurants]",De Forest,"505 W North St\nDe Forest, WI 53532","{u'Monday': {u'close': u'22:00', u'open': u'06...",43.252267,-89.353437,Deforest Family Restaurant,[],True,16,4.0,WI,business
3,"{u'Take-out': True, u'Accepts Credit Cards': T...",LRKJF43s9-3jG9Lgx4zODg,"[Food, Ice Cream & Frozen Yogurt, Fast Food, R...",De Forest,"4910 County Rd V\nDe Forest, WI 53532","{u'Monday': {u'close': u'22:00', u'open': u'10...",43.251045,-89.374983,Culver's,[],True,7,4.5,WI,business
4,"{u'Take-out': True, u'Has TV': False, u'Outdoo...",RgDg-k9S5YD_BaxMckifkg,"[Chinese, Restaurants]",De Forest,"631 S Main St\nDe Forest, WI 53532","{u'Monday': {u'close': u'22:00', u'open': u'11...",43.240875,-89.343722,Chang Jiang Chinese Kitchen,[],True,3,4.0,WI,business


In [22]:
class CityEstimator():
    def __init__(self,):
        import pandas as pd
        pass
    
    def fit(self, X):
        import numpy as np
        self.model=X.groupby("city").apply(np.mean)['stars']
        return self

    def predict(self, x):
        model=self.model
        ##x = x['city']

        if x in model.keys():
            self.y_pred= model[x]
        else:
            self.y_pred = model.median()
        return self.y_pred

#    def score(self, X, y):
#        return # custom score implementation

In [23]:
class CityModelTransformer():
    def __init__(self):
        self.est = CityEstimator()
        self.model = self.est.fit(X)
        return None
    
    def fit(self, X, y):
        return self       
    
    def transform(self,X):
        self.y_preds = []
        if type(X) == dict:
            x = X['city']
            self.y_preds = self.est.predict(x)
        else:        
            for x in X['city']:
                y_pred = [self.est.predict(x)]
                self.y_preds.append(y_pred)         
        return self.y_preds

In [24]:
class ColumnSelectTransformer():
    """
    Returns the columns  of a panda table
    """
    def __init__(self, cols):
        self.cols = cols
        pass

    def fit(self, X, y):
        return self
    
    def transform(self, X):
        if type(X) == dict:
            X = [X[c] for c in self.cols]
        else:
            X = X[self.cols].values
            ##X = X.as_matrix(columns=self.cols)
        return X

In [25]:
class KNNEstimator():
    def __init__(self,n_neighbors, algorithm):
        self.n_neighbors=n_neighbors
        self.algorithm=algorithm
        self.trans = ColumnSelectTransformer(['longitude', 'latitude'])
        pass

    
    def fit(self, X, y):
        from sklearn import neighbors
        self.model = neighbors.KNeighborsRegressor(n_neighbors=self.n_neighbors, algorithm=self.algorithm)
        X = self.trans.transform(X)
        self.model.fit(X,y)
        return self

    def predict(self, x):
        model=self.model
        ##x = (x['longitude'],x['latitude'])
        self.y_pred = model.predict(x)
        return self.y_pred

In [26]:
class LongLatModelTransformer():
    def __init__(self,n):
        self.est = KNNEstimator(n,"auto")
        self.model = self.est.fit(X,y)
        return None
    
    def fit(self, X, y):
        return self    
    
    def transform(self,X):
        self.y_preds = []
        if type(X) == dict:
            x = (X['longitude'], X['latitude'])
            self.y_preds = self.est.predict(x)       
        else:
            for lon,lat in zip(X['longitude'],X['latitude']):
                y_pred = self.est.predict((lon,lat))
                self.y_preds.append(y_pred)         
        return self.y_preds

In [27]:
class CatPreTransformer():
    def __init__(self):
        return None
    
    def fit(self,X):   
        X = X['categories']
        X_trans = []
        for x in X:
            cat_set = {cat:1 for cat in x}
            X_trans.append(cat_set)
        return X_trans
    
    def transform(self, x):
        ##x = x['categories']     
        cat_set = {cat:1 for cat in x}
        return cat_set

In [28]:
class CatTransformer():
    def __init__(self):
        import numpy as np
        from sklearn.feature_extraction import DictVectorizer
        from sklearn.feature_extraction.text import TfidfTransformer
        self.vec = DictVectorizer(sparse=False)
        self.tfid = TfidfTransformer()
        self.trans = CatPreTransformer()
        return None
    
    def fit(self,X):   
        X = self.trans.fit(X)
        X = self.vec.fit_transform(X)
        X = self.tfid.fit_transform(X).toarray()
        return X
 
        
    def transform(self,x):
        x = self.trans.transform(x)
        x = self.vec.transform(x)
        x = self.tfid.transform(x).toarray()
        return x
    
    def colnames(self):
        return self.vec.feature_names_ 

In [488]:
class CatEstimator():
    def __init__(self, alpha):
        self.trans = CatTransformer()    
        self.alpha = alpha
        pass
    
    def fit(self, X, y):    
        from sklearn import linear_model, utils, preprocessing
        X = self.trans.fit(X)
        self.model = linear_model.Ridge(alpha=self.alpha)
        self.model.fit(X,y)
        return self

    def predict(self, x):
        ##x = x['categories']
        x = self.trans.transform(x)
        model=self.model
        self.y_pred = model.predict(x)
        return self.y_pred

In [486]:
class CatModelTransformer():
    def __init__(self,alpha):
        self.est = CatEstimator(alpha)
        self.model = self.est.fit(X,y)

        return None
    
    def fit(self, X, y):
        return self    
    
    def transform(self,X):
        self.y_preds = []
        if type(X) == dict:
            x = X['categories']
            self.y_preds = self.est.predict(x)
        else:
            for x in X['categories']:
                y_pred = self.est.predict(x)
                self.y_preds.append(y_pred)         
        return self.y_preds
  

In [461]:
class AttPreTransformer():
    
    def __init__(self):
        return None
    
    
    def flatten_dict(self, X):

        
        def expand(key, value):
            if isinstance(value, dict):
                return [ (key + '_' + k, v) for k, v in self.flatten_dict(value).items() ]
            else:
                return [ (key, value) ]
        
            
        self.items = [ item for k, v in X.items() for item in expand(k, v) ]

        return dict(self.items)
    
    

In [462]:
class AttTransformer():
    def __init__(self):
        from sklearn.feature_extraction import DictVectorizer
        self.vec = DictVectorizer(sparse=False)
        ##self.trans = ColumnSelectTransformer(['attributes'])  
        self.flat = AttPreTransformer()
        ####self.tfid = TfidfTransformer()
        return None

    
    def fit(self,X):
        X = X['attributes']
        X_pre_vec = []  
        for x in X:
            rec = self.flat.flatten_dict(x)
            cat_set = {cat:1 for cat in rec}
            X_pre_vec.append(cat_set)
        self.X_vec = self.vec.fit_transform(X_pre_vec)
        return self.X_vec
        
    def transform(self,x):
        ##x = x['attributes']   
        rec = self.flat.flatten_dict(x)
        cat_set = {cat:1 for cat in rec}
        self.x_vec = self.vec.transform(cat_set)
        return self.x_vec 
    
     

In [463]:
class AttEstimator():
    def __init__(self, alpha):
        self.trans = AttTransformer()    
        self.alpha = alpha
        pass
    
    def fit(self, X, y):    
        from sklearn import linear_model, utils, preprocessing
        X = self.trans.fit(X)
        self.model = linear_model.Ridge(alpha=self.alpha)
        self.model.fit(X,y)
        return self

    def predict(self, x):
        x = self.trans.transform(x)
        model=self.model    
        self.y_pred = model.predict(x)
        return self.y_pred

In [482]:
class AttModelTransformer():
    def __init__(self,alpha):
        self.est = AttEstimator(alpha)
        self.model = self.est.fit(X,y)
        return None
    
    def fit(self, X, y):
        return self    
    
    def transform(self, X):
        self.y_preds = []
        if type(X) == dict:
            x = X['attributes']
            self.y_preds = self.est.predict(x)
        else:
            for x in X['attributes']:
                y_pred = self.est.predict(x)
                self.y_preds.append(y_pred)         
        return self.y_preds

In [18]:
X = df

In [19]:
y = pd.Series(df['stars'])

In [483]:
att_model2 = AttModelTransformer(0.001)

In [484]:
att_model2.transform(record)

array([ 3.46197872])

In [485]:
att_model2.transform(X)

[array([ 3.88264303]),
 array([ 3.46197872]),
 array([ 3.42785912]),
 array([ 3.08582172]),
 array([ 3.45907344]),
 array([ 3.6839548]),
 array([ 3.6839548]),
 array([ 3.70564707]),
 array([ 3.6839548]),
 array([ 3.51011286]),
 array([ 3.58078703]),
 array([ 3.9131339]),
 array([ 3.46731367]),
 array([ 3.56084567]),
 array([ 3.6233775]),
 array([ 3.48498536]),
 array([ 3.51707724]),
 array([ 3.9131339]),
 array([ 3.55909475]),
 array([ 3.58078703]),
 array([ 4.11182213]),
 array([ 3.50648269]),
 array([ 3.66396148]),
 array([ 3.66396148]),
 array([ 3.74113615]),
 array([ 3.74479275]),
 array([ 3.75969905]),
 array([ 3.45757394]),
 array([ 3.30294396]),
 array([ 3.53892126]),
 array([ 3.6839548]),
 array([ 3.29593958]),
 array([ 3.66761809]),
 array([ 3.49544455]),
 array([ 3.56315107]),
 array([ 3.43006889]),
 array([ 3.6839548]),
 array([ 3.77964041]),
 array([ 3.54449805]),
 array([ 3.51707724]),
 array([ 3.43503985]),
 array([ 3.9131339]),
 array([ 3.74113615]),
 array([ 3.77964041]

In [490]:
cat_model2 = CatModelTransformer(0.001)

In [491]:
cat_model2.transform(X)

[array([ 3.25378542]),
 array([ 3.0740407]),
 array([ 3.20710072]),
 array([ 3.32448623]),
 array([ 3.301926]),
 array([ 3.45715309]),
 array([ 4.10439795]),
 array([ 3.98733608]),
 array([ 4.1077867]),
 array([ 3.41333455]),
 array([ 4.26308164]),
 array([ 3.50196448]),
 array([ 3.27536205]),
 array([ 3.301926]),
 array([ 4.17888047]),
 array([ 4.13575742]),
 array([ 3.52084931]),
 array([ 4.13531314]),
 array([ 4.35857902]),
 array([ 3.46621442]),
 array([ 4.74289306]),
 array([ 3.22113157]),
 array([ 3.95747]),
 array([ 3.49184036]),
 array([ 3.51756621]),
 array([ 4.04304177]),
 array([ 3.78786052]),
 array([ 3.48710153]),
 array([ 3.301926]),
 array([ 3.301926]),
 array([ 4.31802621]),
 array([ 3.28166603]),
 array([ 3.72545007]),
 array([ 3.71012682]),
 array([ 3.46270793]),
 array([ 3.02519258]),
 array([ 4.1077867]),
 array([ 3.28297809]),
 array([ 3.35500632]),
 array([ 3.49529523]),
 array([ 3.28297809]),
 array([ 4.02479068]),
 array([ 3.84863971]),
 array([ 3.28297809]),
 a

In [492]:
cat_model2.transform(record)

array([ 3.0740407])

In [497]:
city_model2 = CityModelTransformer()

In [498]:
city_model2.transform(X)

[[3.6702903946388683],
 [3.75],
 [3.75],
 [3.75],
 [3.75],
 [3.1000000000000001],
 [3.1000000000000001],
 [3.1000000000000001],
 [3.1000000000000001],
 [3.1000000000000001],
 [3.1000000000000001],
 [3.1000000000000001],
 [3.1000000000000001],
 [3.1000000000000001],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112],
 [3.6111111111111112]

In [499]:
city_model2.transform(record)

3.75

In [493]:
longlat_model2 = LongLatModelTransformer(77)

In [494]:
longlat_model2.transform(X)

[array([ 3.87662338]),
 array([ 3.53246753]),
 array([ 3.61038961]),
 array([ 3.64285714]),
 array([ 3.55844156]),
 array([ 3.2987013]),
 array([ 3.42857143]),
 array([ 3.38311688]),
 array([ 3.4025974]),
 array([ 3.38961039]),
 array([ 3.42207792]),
 array([ 3.37662338]),
 array([ 3.38961039]),
 array([ 3.4025974]),
 array([ 3.69480519]),
 array([ 3.73376623]),
 array([ 3.75324675]),
 array([ 3.6038961]),
 array([ 3.51298701]),
 array([ 3.6038961]),
 array([ 3.67532468]),
 array([ 3.69480519]),
 array([ 3.55844156]),
 array([ 3.56493506]),
 array([ 3.57142857]),
 array([ 3.64285714]),
 array([ 3.72727273]),
 array([ 3.68181818]),
 array([ 3.52597403]),
 array([ 3.68831169]),
 array([ 3.66883117]),
 array([ 3.52597403]),
 array([ 3.55844156]),
 array([ 3.55844156]),
 array([ 3.7012987]),
 array([ 3.54545455]),
 array([ 3.76623377]),
 array([ 3.53896104]),
 array([ 3.67532468]),
 array([ 3.62987013]),
 array([ 3.51948052]),
 array([ 3.71428571]),
 array([ 3.71428571]),
 array([ 3.571428

In [495]:
longlat_model2.transform(record)

array([ 3.53246753])

# All models

In [58]:
X_train = df.drop(['stars'], axis=1)

In [59]:
X_train.shape

(37938, 14)

In [60]:
y_train = df['stars']

#  Feature Union

In [500]:
from sklearn import pipeline  
all_features = pipeline.FeatureUnion([
('city_model', CityModelTransformer()),
  ('long_lat_model', LongLatModelTransformer(77)),
        ('cat_model', CatModelTransformer(0.05)),
        ('att_model', AttModelTransformer(0.05))
  ])

In [501]:
from sklearn import linear_model, utils, preprocessing
from sklearn.base import BaseEstimator, RegressorMixin 
from sklearn import neighbors
from sklearn import ensemble
from sklearn.linear_model import LinearRegression

alphas = [0.01,0.05,0.10, 0.50, 1, 5]
k_union = pipeline.Pipeline([
  ("features", all_features),
  ("linreg", linear_model.Ridge(alpha=0.05))
  ])
k_union.fit(X_train, y_train)
##print k_union.score(X_test, y_test)

Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('city_model', <__main__.CityModelTransformer instance at 0x7f8e98866d88>), ('long_lat_model', <__main__.LongLatModelTransformer instance at 0x7f8e98864320>), ('cat_model', <__main__.CatModelTransformer instance at 0x7f8e9886ac68>),....05, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, solver='auto', tol=0.001))])

In [502]:
k_union.predict(record)

array([ 3.03006818])

In [513]:
class EstimatorWrapper():
    def __init__(self, estimator):
        self.estimator = estimator
    
    def fit(self, X, y=None):
        return self
    
    def predict(self, x):
        return self.estimator.predict(x)[0]
    

In [514]:
full_model = EstimatorWrapper(k_union)

In [515]:
import dill
with open('full_model2.dill', 'w') as filename:
    dill.dump(full_model,filename)

In [516]:
record =    {"business_id": "JwUE5GmEO-sH1FuwJgKBlQ", "full_address": "6162 US Highway 51\nDe Forest, WI 53532", "hours": {}, "open": True, "categories": ["Restaurants"], "city": "De Forest", "review_count": 26, "name": "Pine Cone Restaurant", "neighborhoods": [], "longitude": -89.335843999999994, "state": "WI", "stars": 4.0, "latitude": 43.238892999999997, "attributes": {"Take-out": True, "Good For": {"dessert": False, "latenight": False, "lunch": True, "dinner": False, "breakfast": False, "brunch": False}, "Caters": False, "Noise Level": "average", "Takes Reservations": False, "Delivery": False, "Ambience": {"romantic": False, "intimate": False, "touristy": False, "hipster": False, "divey": False, "classy": False, "trendy": False, "upscale": False, "casual": False}, "Parking": {"garage": False, "street": False, "validated": False, "lot": True, "valet": False}, "Has TV": True, "Outdoor Seating": False, "Attire": "casual", "Alcohol": "none", "Waiter Service": True, "Accepts Credit Cards": True, "Good for Kids": True, "Good For Groups": True, "Price Range": 1}, "type": "business"}
print record

{'business_id': 'JwUE5GmEO-sH1FuwJgKBlQ', 'full_address': '6162 US Highway 51\nDe Forest, WI 53532', 'hours': {}, 'open': True, 'categories': ['Restaurants'], 'city': 'De Forest', 'review_count': 26, 'name': 'Pine Cone Restaurant', 'neighborhoods': [], 'longitude': -89.335844, 'state': 'WI', 'stars': 4.0, 'latitude': 43.238893, 'attributes': {'Take-out': True, 'Price Range': 1, 'Outdoor Seating': False, 'Caters': False, 'Noise Level': 'average', 'Parking': {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}, 'Delivery': False, 'Attire': 'casual', 'Has TV': True, 'Good For': {'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': False, 'breakfast': False}, 'Takes Reservations': False, 'Ambience': {'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'touristy': False, 'casual': False}, 'Waiter Service': True, 'Accepts Credit Cards': True, 'Good for Kids': True

In [517]:
full_model.predict(record)

3.0300681815273749

In [518]:
record = {"business_id": "vcNAWiLM4dR7D2nwwJ7nCA", "full_address": "4840 E Indian School Rd\nSte 101\nPhoenix, AZ 85018", "hours": {"Tuesday": {"close": "17:00", "open": "08:00"}, "Friday": {"close": "17:00", "open": "08:00"}, "Monday": {"close": "17:00", "open": "08:00"}, "Wednesday": {"close": "17:00", "open": "08:00"}, "Thursday": {"close": "17:00", "open": "08:00"}}, "open": True, "categories": ["Doctors", "Health & Medical"], "city": "Phoenix", "review_count": 7, "name": "Eric Goldberg, MD", "neighborhoods": [], "longitude": -111.98375799999999, "state": "AZ", "stars": 3.5, "latitude": 33.499313000000001, "attributes": {"By Appointment Only": True}, "type": "business"}
print record

{'business_id': 'vcNAWiLM4dR7D2nwwJ7nCA', 'full_address': '4840 E Indian School Rd\nSte 101\nPhoenix, AZ 85018', 'hours': {'Thursday': {'close': '17:00', 'open': '08:00'}, 'Tuesday': {'close': '17:00', 'open': '08:00'}, 'Friday': {'close': '17:00', 'open': '08:00'}, 'Wednesday': {'close': '17:00', 'open': '08:00'}, 'Monday': {'close': '17:00', 'open': '08:00'}}, 'open': True, 'categories': ['Doctors', 'Health & Medical'], 'city': 'Phoenix', 'review_count': 7, 'name': 'Eric Goldberg, MD', 'neighborhoods': [], 'longitude': -111.983758, 'state': 'AZ', 'stars': 3.5, 'latitude': 33.499313, 'attributes': {'By Appointment Only': True}, 'type': 'business'}


In [519]:
full_model.predict(record)

3.4689162877961697

In [ ]:
CatTransformer

In [5]:
len(data)

37938

In [49]:
cat = CatPreTransformer()

In [50]:
new_df = cat.fit(df)

In [51]:
new_df

[{u'a': 1,
  u'c': 1,
  u'e': 1,
  u'g': 1,
  u'i': 1,
  u'o': 1,
  u'r': 1,
  u's': 1,
  u't': 1},
 {u'_': 1, u'b': 1, u'd': 1, u'e': 1, u'i': 1, u'n': 1, u's': 1, u'u': 1}]

In [43]:
class CatPreTransformer():
    def __init__(self):
        return None
    
    def fit(self,X):   
        X = X[['categories', 'business_id']]
        X_trans = []
        for x in X:
            cat_set = {cat:1 for cat in x}
            X_trans.append(cat_set)
        return X_trans
    
    def transform(self, x):
        ##x = x['categories']     
        cat_set = {cat:1 for cat in x}
        return cat_set

In [52]:
df

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
0,{u'By Appointment Only': True},vcNAWiLM4dR7D2nwwJ7nCA,"[Doctors, Health & Medical]",Phoenix,"4840 E Indian School Rd\nSte 101\nPhoenix, AZ ...","{u'Thursday': {u'close': u'17:00', u'open': u'...",33.499313,-111.983758,"Eric Goldberg, MD",[],True,7,3.5,AZ,business
1,"{u'Take-out': True, u'Price Range': 1, u'Outdo...",JwUE5GmEO-sH1FuwJgKBlQ,[Restaurants],De Forest,"6162 US Highway 51\nDe Forest, WI 53532",{},43.238893,-89.335844,Pine Cone Restaurant,[],True,26,4.0,WI,business
2,"{u'Take-out': True, u'Outdoor Seating': False,...",uGykseHzyS5xAMWoN6YUqA,"[American (Traditional), Restaurants]",De Forest,"505 W North St\nDe Forest, WI 53532","{u'Monday': {u'close': u'22:00', u'open': u'06...",43.252267,-89.353437,Deforest Family Restaurant,[],True,16,4.0,WI,business
3,"{u'Take-out': True, u'Accepts Credit Cards': T...",LRKJF43s9-3jG9Lgx4zODg,"[Food, Ice Cream & Frozen Yogurt, Fast Food, R...",De Forest,"4910 County Rd V\nDe Forest, WI 53532","{u'Monday': {u'close': u'22:00', u'open': u'10...",43.251045,-89.374983,Culver's,[],True,7,4.5,WI,business
4,"{u'Take-out': True, u'Has TV': False, u'Outdoo...",RgDg-k9S5YD_BaxMckifkg,"[Chinese, Restaurants]",De Forest,"631 S Main St\nDe Forest, WI 53532","{u'Monday': {u'close': u'22:00', u'open': u'11...",43.240875,-89.343722,Chang Jiang Chinese Kitchen,[],True,3,4.0,WI,business
5,{},oLctHIA1AxmsgOuu4dM6Vw,"[Television Stations, Mass Media]",Mc Farland,"4156 County Rd B\nMc Farland, WI 53558",{},42.968507,-89.322920,Charter Communications,[],False,10,1.5,WI,business
6,{},ZW2WeP2Hp20tq0RG1NFkoQ,"[Home Services, Heating & Air Conditioning/HVAC]",Mc Farland,"4709 Triangle St\nMc Farland, WI 53558",{},43.030356,-89.298544,Air Quality Systems,[],True,4,2.0,WI,business
7,{u'Wi-Fi': u'free'},95p9Xg358BezJyk1wqzzyg,"[Libraries, Public Services & Government]",Mc Farland,"5114 Farwell St\nMc Farland, WI 53558","{u'Monday': {u'close': u'20:00', u'open': u'09...",43.014093,-89.294175,McFarland Public Library,[],True,4,2.5,WI,business
8,{},ZlhAvs_6ryi4_efZ1SpMbg,"[Veterinarians, Pets]",Mc Farland,"4811 Larson Beach Rd\nMc Farland, WI 53558","{u'Monday': {u'close': u'18:00', u'open': u'07...",43.017124,-89.299119,Spartan Animal Hospital,[],True,3,4.5,WI,business
9,"{u'Alcohol': u'full_bar', u'Noise Level': u'lo...",_wZTYYL7cutanzAnJUTGMA,"[Bars, American (Traditional), Nightlife, Loun...",Mc Farland,"4506 Larson Beach Rd\nMc Farland, WI 53558",{},43.017701,-89.303789,Beach House Restaurant & Lounge,[],True,31,3.5,WI,business


In [53]:
df[['categories', 'business_id']]

,categories,business_id
0,"[Doctors, Health & Medical]",vcNAWiLM4dR7D2nwwJ7nCA
1,[Restaurants],JwUE5GmEO-sH1FuwJgKBlQ
2,"[American (Traditional), Restaurants]",uGykseHzyS5xAMWoN6YUqA
3,"[Food, Ice Cream & Frozen Yogurt, Fast Food, R...",LRKJF43s9-3jG9Lgx4zODg
4,"[Chinese, Restaurants]",RgDg-k9S5YD_BaxMckifkg
5,"[Television Stations, Mass Media]",oLctHIA1AxmsgOuu4dM6Vw
6,"[Home Services, Heating & Air Conditioning/HVAC]",ZW2WeP2Hp20tq0RG1NFkoQ
7,"[Libraries, Public Services & Government]",95p9Xg358BezJyk1wqzzyg
8,"[Veterinarians, Pets]",ZlhAvs_6ryi4_efZ1SpMbg
9,"[Bars, American (Traditional), Nightlife, Loun...",_wZTYYL7cutanzAnJUTGMA


In [62]:
class RestTransformer():
    def __init__(self):
        return None
    
    def fit(self,X):   
        X = X[['categories', 'business_id']]
        X_list = []
        for x in X:
            if 'Restaurants'in x:
                X_list.append(True)
            else:
                X_list.append(False)
        return X
    
    def transform(self, x):
        ##x = x['categories']     
        cat_set = {cat:1 for cat in x}
        return cat_set

In [63]:
rest = RestTransformer()

In [65]:
new_df = rest.fit(df)

In [58]:
new_df['Restaurants']= new_df['categories']


[False, False]

In [67]:
list_categories = new_df['categories']

In [68]:
list_restaurants = []
for items in list_categories:
    if 'Restaurants'in items:
        list_restaurants.append(True)
    else:
        list_restaurants.append(False)
        


In [69]:
list_restaurants

[False,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,

In [71]:
new_df['rest'] = list_restaurants

/opt/conda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [72]:
new_df

,categories,business_id,rest
0,"[Doctors, Health & Medical]",vcNAWiLM4dR7D2nwwJ7nCA,False
1,[Restaurants],JwUE5GmEO-sH1FuwJgKBlQ,True
2,"[American (Traditional), Restaurants]",uGykseHzyS5xAMWoN6YUqA,True
3,"[Food, Ice Cream & Frozen Yogurt, Fast Food, R...",LRKJF43s9-3jG9Lgx4zODg,True
4,"[Chinese, Restaurants]",RgDg-k9S5YD_BaxMckifkg,True
5,"[Television Stations, Mass Media]",oLctHIA1AxmsgOuu4dM6Vw,False
6,"[Home Services, Heating & Air Conditioning/HVAC]",ZW2WeP2Hp20tq0RG1NFkoQ,False
7,"[Libraries, Public Services & Government]",95p9Xg358BezJyk1wqzzyg,False
8,"[Veterinarians, Pets]",ZlhAvs_6ryi4_efZ1SpMbg,False
9,"[Bars, American (Traditional), Nightlife, Loun...",_wZTYYL7cutanzAnJUTGMA,True


In [78]:
df_rest = new_df.groupby('business_id').head(1)

In [79]:
df_rest

,categories,business_id,rest
0,"[Doctors, Health & Medical]",vcNAWiLM4dR7D2nwwJ7nCA,False
1,[Restaurants],JwUE5GmEO-sH1FuwJgKBlQ,True
2,"[American (Traditional), Restaurants]",uGykseHzyS5xAMWoN6YUqA,True
3,"[Food, Ice Cream & Frozen Yogurt, Fast Food, R...",LRKJF43s9-3jG9Lgx4zODg,True
4,"[Chinese, Restaurants]",RgDg-k9S5YD_BaxMckifkg,True
5,"[Television Stations, Mass Media]",oLctHIA1AxmsgOuu4dM6Vw,False
6,"[Home Services, Heating & Air Conditioning/HVAC]",ZW2WeP2Hp20tq0RG1NFkoQ,False
7,"[Libraries, Public Services & Government]",95p9Xg358BezJyk1wqzzyg,False
8,"[Veterinarians, Pets]",ZlhAvs_6ryi4_efZ1SpMbg,False
9,"[Bars, American (Traditional), Nightlife, Loun...",_wZTYYL7cutanzAnJUTGMA,True


In [81]:
df_rest = df_rest[(df_rest['rest']==True)]

In [82]:
df_rest

,categories,business_id,rest
1,[Restaurants],JwUE5GmEO-sH1FuwJgKBlQ,True
2,"[American (Traditional), Restaurants]",uGykseHzyS5xAMWoN6YUqA,True
3,"[Food, Ice Cream & Frozen Yogurt, Fast Food, R...",LRKJF43s9-3jG9Lgx4zODg,True
4,"[Chinese, Restaurants]",RgDg-k9S5YD_BaxMckifkg,True
9,"[Bars, American (Traditional), Nightlife, Loun...",_wZTYYL7cutanzAnJUTGMA,True
12,"[Pizza, Restaurants]",zOc8lbjViUZajbY7M0aUCQ,True
13,"[Chinese, Restaurants]",UgjVZTSOaYoEvws_lAP_Dw,True
15,"[Party & Event Planning, Asian Fusion, Event P...",SKLw05kEIlZcpTD5pqma8Q,True
16,"[Mexican, Restaurants]",77ESrCo7hQ96VpCWWdvoxg,True
21,"[Sandwiches, Pizza, Chicken Wings, Restaurants]",KTqNU4plO23583DYAMGXYg,True
